ENM 5310 Project - Michael Chiou, Stepehen Ching, Quan Vo

flax_CNN.ipynb template from class github

In [ ]:
!pip install typing Tuble

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 4.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
ERROR: Ignored the following versions that require a different python version: 3.10.0.0 Requires-Python >=2.7, !=3.0.*, !=3.1.*, !=3.2.*, !=3.3.*, <3.5; 3.7.4.2 Requires-Python >=2.7, !=3.0.*, !=3.1.*, !=3.2.*, !=3.3.*, <3.5
ERROR: Could not find a version that satisfies the requirement Tuble (from versions: none)
ERROR: No matching distribution found for Tuble


In [ ]:
# In this block, modify the imported data to be rotated randomly to get training data
# random distribution of 0-180 degrees, apply that rotation to the image and the ground truth angle
# i.e. original ground truth vector, rotated by rotation matrix of angle
# if angle of unit vector is > 180 or < 0, reflect over x-axis and y-axis to get equivalent/new ground truth

In [ ]:
import os
import glob
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
import matplotlib.pyplot as plt
import tifffile as tiff
import jax.numpy as jnp
from jax import random, vmap, grad, jit
from jax.nn import one_hot
from PIL import Image
import optax
from sklearn.metrics import confusion_matrix
from typing import Any, Callable, Sequence, Optional, Union, Dict

# libraries that we need for image import and processing
import skimage
import re
from pathlib import Path
from skimage import io
from skimage import filters
from skimage import data
from scipy.ndimage import rotate
# connect w/ the files in drive
from google.colab import drive
if not os.path.exists('/content/drive'):
    drive.mount('/content/drive')


**Setup parameters and connect to the data in the shared drive**

In [ ]:
import os
import glob
import numpy as np
import pandas as pd
import tifffile as tiff
from tqdm import tqdm
import random
from sklearn.model_selection import train_test_split

# ==========================================
# 1. CONFIGURATION
# ==========================================
BASE_DIR = '/content/drive/Shareddrives/ENM 5310 Group Project'
SOURCE_DIR = os.path.join(BASE_DIR, 'Training Data')
OUTPUT_DIR = '/content/drive/Shareddrives/ENM 5310 Group Project/Augmented Data'
MODEL_SAVE_DIR = os.path.join(BASE_DIR, 'Models')
CSV_FILENAME = 'fibril_orientation_2d_results.csv'

for d in [OUTPUT_DIR, MODEL_SAVE_DIR]:
    if not os.path.exists(d):
        os.makedirs(d)

CROP_SIZE = (256, 256)   # (H, W)
CROPS_PER_ROTATION = 5
ROTATIONS = [0, 90, 180, 270]

if not os.path.exists(OUTPUT_DIR):
    os.makedirs(OUTPUT_DIR)

# ==========================================
# 2. HELPER FUNCTIONS
# ==========================================
def parse_pos(pos_str):
    try:
        # User specified format: (y, x, 0)
        coords = [int(c) for c in pos_str.replace('(', '').replace(')', '').split(',')]
        return coords[0], coords[1] # returns y, x
    except:
        return None, None

def rotate_coords_and_vectors(x, y, u, v, img_w, img_h, angle):
    """
    Rotates coordinates and vectors.
    Inputs: x (col), y (row), u (x-vec), v (y-vec)
    Returns: new_x, new_y, new_u, new_v, new_h, new_w
    """
    if angle == 0:
        return x, y, u, v, img_h, img_w

    elif angle == 90:
        # 90 deg Counter-Clockwise
        # Dimensions swap: New W = Old H, New H = Old W
        new_w = img_h
        new_h = img_w

        # Pixel Rotation: (x, y) -> (y, W - 1 - x)
        # Note: In array coords, this maps A[y, x] -> A'[new_y, new_x]
        # Visual 90 deg CCW:
        # new_x (col) = y (old row)
        # new_y (row) = (Width - 1) - x (old col)
        new_x = y
        new_y = (img_w - 1) - x

        # Vector Rotation 90 CCW:
        # (1,0) Right -> (0,-1) Up (Negative V is Up in image coords)
        # (0,1) Down  -> (1,0) Right
        # u' = v, v' = -u
        new_u = v
        new_v = -u
        return new_x, new_y, new_u, new_v, new_h, new_w

    elif angle == 180:
        new_w, new_h = img_w, img_h
        # 180 deg
        new_x = img_w - 1 - x
        new_y = img_h - 1 - y
        new_u = -u
        new_v = -v
        return new_x, new_y, new_u, new_v, new_h, new_w

    elif angle == 270:
        # 270 deg CCW (or 90 CW)
        new_w = img_h
        new_h = img_w

        # Pixel: (x, y) -> (H - 1 - y, x)
        new_x = (img_h - 1) - y
        new_y = x

        # Vector: 90 CW
        # (1,0) Right -> (0,1) Down
        # (0,1) Down  -> (-1,0) Left
        # u' = -v, v' = u
        new_u = -v
        new_v = u
        return new_x, new_y, new_u, new_v, new_h, new_w

def generate_blurred_data(file_list, df, output_subdir):
  "Generate bilteral filter images"
  "This does not change the ground truth"
  save_path = os.path.join(OUTPUT_DIR, output_subdir)
  if not os.path.exists(save_path):
    os.makedirs(save_path
# ==========================================
# 3. MAIN LOOP
# ==========================================
csv_path = os.path.join(SOURCE_DIR, CSV_FILENAME)
df = pd.read_csv(csv_path)
df['clean_filename'] = df.iloc[:, 0].apply(lambda x: os.path.basename(str(x)).strip())

source_files = sorted(glob.glob(os.path.join(SOURCE_DIR, 'training*.tiff')))
new_csv_rows = []

print(f"Generating augmented data from {len(source_files)} images...")

for file_path in tqdm(source_files):
    filename = os.path.basename(file_path)
    base_name = os.path.splitext(filename)[0]

    try:
        img = tiff.imread(file_path) # (H, W)
    except:
        continue

    img_labels = df[df['clean_filename'] == filename]

    for angle in ROTATIONS:
        # A. Rotate Image
        k = angle // 90
        rot_img = np.rot90(img, k=k)
        curr_h, curr_w = rot_img.shape

        # B. Rotate Vectors
        rotated_labels = []
        for _, row in img_labels.iterrows():
            pos_y, pos_x = parse_pos(row.iloc[1]) # User said: (y, x, 0)
            if pos_x is None: continue

            # Pass original W, H for coordinate math
            orig_h, orig_w = img.shape

            rx, ry, ru, rv, _, _ = rotate_coords_and_vectors(
                pos_x, pos_y, row.iloc[2], row.iloc[3], orig_w, orig_h, angle
            )
            rotated_labels.append({'x': rx, 'y': ry, 'u': ru, 'v': rv})

        # C. Random Crops
        for i in range(CROPS_PER_ROTATION):
            if curr_h < CROP_SIZE[0] or curr_w < CROP_SIZE[1]: continue

            y_start = random.randint(0, curr_h - CROP_SIZE[0])
            x_start = random.randint(0, curr_w - CROP_SIZE[1])

            crop_img = rot_img[y_start:y_start+CROP_SIZE[0], x_start:x_start+CROP_SIZE[1]]

            new_filename = f"{base_name}_rot{angle}_crop{i}.tiff"
            tiff.imwrite(os.path.join(OUTPUT_DIR, new_filename), crop_img)

            # Filter labels
            for label in rotated_labels:
                lx, ly = label['x'], label['y']
                if x_start <= lx < x_start + CROP_SIZE[1] and y_start <= ly < y_start + CROP_SIZE[0]:
                    new_x = lx - x_start
                    new_y = ly - y_start

                    new_csv_rows.append({
                        'filename': new_filename,
                        'pos': f"({new_y},{new_x},0)", # Write back as (y, x, 0)
                        'u': label['u'],
                        'v': label['v']
                    })

if len(new_csv_rows) > 0:
    aug_df = pd.DataFrame(new_csv_rows)
    aug_df.to_csv(os.path.join(OUTPUT_DIR, 'augmented_labels.csv'), index=False)
    print("Synthesis Complete.")

Generating augmented data from 12 images...


 17%|█▋        | 2/12 [00:54<04:32, 27.23s/it]


KeyboardInterrupt: 

**check the raw images**

In [ ]:
# 2. RAW IMAGE CHECK
# ==========================================
def check_raw_images(file_list, num_to_show=3):
    print(f"--- Checking first {num_to_show} raw images ---")
    plt.figure(figsize=(18, 6))

    for i in range(min(len(file_list), num_to_show)):
        path = file_list[i]
        try:
            # Load with tifffile to preserve 16-bit/32-bit data
            img = tiff.imread(path)

            # Normalize for display (Min-Max scaling to 0-1)
            img_disp = (img - img.min()) / (img.max() - img.min())

            plt.subplot(1, num_to_show, i+1)
            plt.imshow(img_disp, cmap='gray') # Use gray for scientific viz
            plt.title(f"{os.path.basename(path)}\nShape: {img.shape}\nRange: [{img.min()}, {img.max()}]")
            plt.axis('off')
        except Exception as e:
            print(f"Error loading {path}: {e}")

    plt.tight_layout()
    plt.show()
    print("---------------------------------------------")

**Data Augmentation Class (rotations)**

In [ ]:
class PatchRotationAugmentation:
  """applies rotation transformation to 120x120 patches and the vectors/angle"""

  def = __init__(self, apply_augmentation: bool = True, angle_range: Tuple[float, float]  = (0, 180)):
    """
        Args:
            apply_augmentation: if True, applies random rotation
            angle_range: (min, max) ration angle in degrees
    """
    self.apply_augmentation = apply_augmentation
    self.angle_range = angle_range

  def calcualte_max_inscribed_square(self, patch_size: int) -> int:
    """
        Return the size given a max crop of 45 degrees, apply to all patches for consistency
        Args:
            patch_size: Original patch size (nxn)
    """
    return int(patch_size / np.sqrt(2))

  def rotate_patch(self, patch: np.ndarray, angle_deg: float) -> np.ndarray:
    """
        Args:
            patch: (H, W) - ideally square
            angle_deg: rotation transform in angles
        Returns:
            rotated and uniformly cropped patch
    """
    original_size = patch.shape[0]
    rotated = rotate(patch, angle_deg, reshape=True, order=1, mode='constant', cval=0)
    # scipy.ndimage.rotate(), reshape=false ensures the rotated image is mapped onto the orginal image size (120x120), with black corners, the irrelevant corners of the original image are also automatically cropped
    # from the new size, select the rows and columns of the rotated image which describe the cropped image for a consistently shaped image of ~(84x84)
    inscribed_size = self.calculate_max_inscribed_square(original_size)
    new_h, new_w = rotated.shape
    start_h = (new_h - inscribed_size) // 2
    start_w = (new_w - inscribed_size) // 2
    cropped = rotated[start_h:start_h+inscribed_size, start_w:start_w+inscribed_size]
    return

  def adjust_angle(self, theta: float, rotation_def: float) -> float:
    """
        Args:
            theta: original ground truth orientation angle in radians
            rotation_deg: applied rotation in degrees
        Returns:
            adjusted angle in radians, normalized to [-pi/2, pi/2]
    """
    rotation_rad = np.radians(rotation_deg)
    # Add rotation to original angle
    new_theta = theta + rotation_rad
    # Normalize to [-pi/2, pi/2]
    new_theta = np.arctan(np.tan(new_theta))
    return new_theta

  def __call__(self, patch: np.ndarray, theta: float) -> Tuple[np.ndarray, float, float]:
    """
        Apply rotation augmentation to a patch + its ground truth angle
        Args:
            patch: (H, W) - ideally square
            theta: original ground truth orientation angle in radians
        Returns:
            (rotated_patch, adjusted_theta, applied rotation_deg)
    """
    if not self.apply_augmentation
      return patch, theta, 0.0
    rotation_deg = np.random.uniform(self.angle_range[0], self.angle_range[1])
    rotated_patch = self.rotate_patch(patch, rotation_deg)
    adjusted_theta = self.adjust_angle(theta, rotation_deg)
    return rotated_patch, adjusted_theta, rotation_deg

#Final Code:

In [ ]:
import os
import glob
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
import matplotlib.pyplot as plt
import tifffile as tiff
from skimage import exposure
import re

# 1. GLOBAL CONFIGURATION & SETUP
from google.colab import drive
if not os.path.exists('/content/drive'):
    drive.mount('/content/drive')

# Paths
BASE_DIR = '/content/drive/Shareddrives/ENM 5310 Group Project'
ORIGINAL_DATA_DIR = os.path.join(BASE_DIR, 'Training Data')
AUGMENTED_DATA_DIR = '/content/drive/Shareddrives/ENM 5310 Group Project/Augmented Data'
SAVE_DIR = '/content/drive/Shareddrives/ENM 5310 Group Project/Models Evaluation'

if not os.path.exists(SAVE_DIR):
    os.makedirs(SAVE_DIR)

# Hyperparameters
BATCH_SIZE = 16
LR = 5e-4
EPOCHS = 20
REGION_SIZE = 100
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'

# Experiment Grid
CLAHE_OPTS = [True, False]
LOSS_OPTS = ['cosine', 'mse']
DEPTH_OPTS = ['deep', 'shallow']

# 2. DATASET CLASS
class CollagenDataset(Dataset):
    def __init__(self, image_files, csv_df, use_clahe):
        self.image_files = image_files
        self.csv_df = csv_df
        self.use_clahe = use_clahe
        self.csv_df['clean_filename'] = self.csv_df.iloc[:, 0].apply(lambda x: os.path.basename(str(x)).strip())
        self.img_transform = transforms.Compose([transforms.ToTensor()])

    def __len__(self):
        return len(self.image_files)

    def _generate_gt_map(self, filename, shape):
        clean_name = os.path.basename(filename).strip()
        file_data = self.csv_df[self.csv_df['clean_filename'] == clean_name]

        acc_u = np.zeros(shape, dtype=np.float32)
        acc_v = np.zeros(shape, dtype=np.float32)
        mask = np.zeros(shape, dtype=np.float32)
        half = REGION_SIZE // 2

        for _, row in file_data.iterrows():
            pos_str = row.iloc[1]
            try:
                coords = [int(c) for c in pos_str.replace('(', '').replace(')', '').split(',')]
                cx, cy = coords[0], coords[1]

                # Bounds check
                y0, y1 = cy - half, cy + half
                x0, x1 = cx - half, cx + half
                if not (x0 >= 0 and x1 <= shape[1] and y0 >= 0 and y1 <= shape[0]): continue

                if 'u' in row: u = float(row['u']); v = float(row['v'])
                else: u = float(row.iloc[2]); v = float(row.iloc[3])

                mag = np.sqrt(u**2 + v**2)
                if mag > 1e-6: u/=mag; v/=mag
                if u < 0: u = -u; v = -v

                acc_u[y0:y1, x0:x1] += u
                acc_v[y0:y1, x0:x1] += v
                mask[y0:y1, x0:x1] = 1.0
            except: continue

        theta = np.arctan(acc_v / (acc_u + 1e-8))
        target_sin = np.sin(2 * theta)
        target_cos = np.cos(2 * theta)

        target_sin[mask == 0] = 0
        target_cos[mask == 0] = 0

        gt_map = np.stack([target_sin, target_cos], axis=0)
        return gt_map, np.expand_dims(mask, axis=0)

    def __getitem__(self, idx):
        path = self.image_files[idx]
        filename = os.path.basename(path)

        image = tiff.imread(path).astype(np.float32)

        # Min-Max
        if image.max() > image.min():
            image = (image - image.min()) / (image.max() - image.min())

        # CLAHE
        if self.use_clahe:
            image = exposure.equalize_adapthist(image, kernel_size=None, clip_limit=0.03)
            image = image.astype(np.float32)

        if len(image.shape) == 2: image = np.expand_dims(image, axis=0)
        image_tensor = torch.from_numpy(image)

        gt_map, mask = self._generate_gt_map(filename, (image.shape[1], image.shape[2]))

        return image_tensor, torch.from_numpy(gt_map), torch.from_numpy(mask), idx

# 3. CONFIGURABLE MODEL
class ResidualBlock(nn.Module):
    def __init__(self, in_c, out_c):
        super().__init__()
        self.conv = nn.Sequential(
            nn.BatchNorm2d(in_c), nn.ReLU(), nn.Conv2d(in_c, out_c, 3, 1, 1),
            nn.BatchNorm2d(out_c), nn.ReLU(), nn.Conv2d(out_c, out_c, 3, 1, 1)
        )
        self.short = nn.Conv2d(in_c, out_c, 1) if in_c != out_c else nn.Identity()
    def forward(self, x): return self.short(x) + self.conv(x)

class ConfigurableUNet(nn.Module):
    def __init__(self, depth='deep'):
        super().__init__()
        self.depth = depth
        self.inc = nn.Sequential(nn.Conv2d(1, 32, 3, 1, 1), nn.ReLU(), nn.Conv2d(32, 32, 3, 1, 1))

        self.down1 = nn.Sequential(nn.MaxPool2d(2), ResidualBlock(32, 64))
        self.down2 = nn.Sequential(nn.MaxPool2d(2), ResidualBlock(64, 128))

        if self.depth == 'deep':
            self.down3 = nn.Sequential(nn.MaxPool2d(2), ResidualBlock(128, 256))
            self.up2 = nn.ConvTranspose2d(256, 128, 2, 2)
            self.conv2 = ResidualBlock(256, 128)

        self.up3 = nn.ConvTranspose2d(128, 64, 2, 2)
        self.conv3 = ResidualBlock(128, 64)
        self.up4 = nn.ConvTranspose2d(64, 32, 2, 2)
        self.conv4 = ResidualBlock(64, 32)
        self.outc = nn.Conv2d(32, 2, 1)

    def forward(self, x):
        x1 = self.inc(x)
        x2 = self.down1(x1)
        x3 = self.down2(x2)

        if self.depth == 'deep':
            x4 = self.down3(x3)
            x = self.up2(x4)
            dY = x3.size(2)-x.size(2); dX = x3.size(3)-x.size(3)
            x = nn.functional.pad(x, [dX//2, dX-dX//2, dY//2, dY-dY//2])
            x = self.conv2(torch.cat([x3, x], 1))
        else:
            x = x3

        x = self.up3(x)
        dY = x2.size(2)-x.size(2); dX = x2.size(3)-x.size(3)
        x = nn.functional.pad(x, [dX//2, dX-dX//2, dY//2, dY-dY//2])
        x = self.conv3(torch.cat([x2, x], 1))

        x = self.up4(x)
        dY = x1.size(2)-x.size(2); dX = x1.size(3)-x.size(3)
        x = nn.functional.pad(x, [dX//2, dX-dX//2, dY//2, dY-dY//2])
        x = self.conv4(torch.cat([x1, x], 1))

        return F.normalize(self.outc(x), p=2, dim=1)

# 4. EVALUATION UTILITIES
def get_shortest_angle_diff(angle1, angle2):
    diff = np.abs(angle1 - angle2)
    diff = diff % 180
    diff = np.minimum(diff, 180 - diff)
    return diff

def evaluate_and_plot(model, loader, model_name):
    model.eval()
    all_gt, all_pred = [], []

    plot_saved = False

    with torch.no_grad():
        for i, (images, _, _, indices) in enumerate(loader):
            outputs = model(images.to(DEVICE))

            idx = indices[0].item()
            path = loader.dataset.image_files[idx]
            filename = os.path.basename(path)

            # Prediction Map
            sin_map = outputs[0, 0].cpu().numpy()
            cos_map = outputs[0, 1].cpu().numpy()
            pred_theta = 0.5 * np.arctan2(sin_map, cos_map)

            # Extract Vectors
            file_data = loader.dataset.csv_df[loader.dataset.csv_df['clean_filename'] == filename]
            X, Y, Ug, Vg, Up, Vp = [], [], [], [], [], []
            h, w = pred_theta.shape

            for _, row in file_data.iterrows():
                try:
                    c = [int(x) for x in row.iloc[1].replace('(','').replace(')','').split(',')]
                    cx, cy = c[0], c[1]
                    half = REGION_SIZE // 2
                    if not (cx-half>=0 and cx+half<=w and cy-half>=0 and cy+half<=h): continue

                    if 'u' in row: u, v = float(row['u']), float(row['v'])
                    else: u, v = float(row.iloc[2]), float(row.iloc[3])

                    mag = np.sqrt(u**2 + v**2)
                    if mag > 1e-6: u/=mag; v/=mag
                    if u < 0: u, v = -u, -v

                    gt_deg = np.degrees(np.arctan2(v, u + 1e-8))

                    th = pred_theta[cy, cx]
                    pd_deg = np.degrees(th)
                    pu, pv = np.cos(th), np.sin(th)

                    all_gt.append(gt_deg)
                    all_pred.append(pd_deg)

                    X.append(cx); Y.append(cy)
                    Ug.append(u); Vg.append(v)
                    Up.append(pu); Vp.append(pv)
                except: continue

            if not plot_saved and len(X) > 0:
                img_np = images[0, 0].cpu().numpy()
                fig, ax = plt.subplots(1, 2, figsize=(16, 6))
                ax[0].imshow(img_np, cmap='gray')
                ax[0].quiver(X, Y, Ug, Vg, color='lime', scale=20, pivot='mid')
                ax[0].set_title(f"Ground Truth ({filename})")

                ax[1].imshow(img_np, cmap='gray')
                ax[1].quiver(X, Y, Up, Vp, color='red', scale=20, pivot='mid')
                ax[1].set_title(f"Prediction: {model_name}")

                plt.tight_layout()
                plt.savefig(os.path.join(SAVE_DIR, f"{model_name}_Prediction.png"))
                plt.close(fig)
                plot_saved = True

    # Calculate Metrics
    gt_arr = np.array(all_gt)
    pd_arr = np.array(all_pred)

    errors = get_shortest_angle_diff(gt_arr, pd_arr)
    mae = np.mean(errors)
    success = np.mean(errors < 15.0) * 100
    spread = np.std(pd_arr) / (np.std(gt_arr) + 1e-8)
    bias = np.mean(pd_arr) - np.mean(gt_arr)

    return [mae, success, spread, bias]

# 5. MAIN EXPERIMENT LOOP
# Load Files
try: df_tr = pd.read_csv(os.path.join(AUGMENTED_DATA_DIR, 'augmented_labels.csv'))
except: df_tr = pd.DataFrame()
tr_files = sorted(glob.glob(os.path.join(AUGMENTED_DATA_DIR, '*.tiff')))

try: df_te = pd.read_csv(os.path.join(ORIGINAL_DATA_DIR, 'fibril_orientation_2d_results.csv'))
except: df_te = pd.DataFrame()
all_orig = sorted(glob.glob(os.path.join(ORIGINAL_DATA_DIR, 'training*.tiff')))
te_files = [f for f in all_orig if int(re.findall(r'\d+', os.path.basename(f))[0]) >= 10]

FINAL_RESULTS = {}

print(f"--- STARTING 8-MODEL COMPARISON ---")
print(f"Results are saved to: {SAVE_DIR}")

run_count = 1
total_runs = 8

for clahe in CLAHE_OPTS:
    for loss_type in LOSS_OPTS:
        for depth in DEPTH_OPTS:

            model_name = f"CLAHE-{clahe}_{loss_type.upper()}_{depth}"
            print(f"\n[{run_count}/{total_runs}] Running: {model_name}")

            # 1. Setup Data with specific CLAHE setting
            tr_set = CollagenDataset(tr_files, df_tr, use_clahe=clahe)
            te_set = CollagenDataset(te_files, df_te, use_clahe=clahe)
            tr_load = DataLoader(tr_set, batch_size=BATCH_SIZE, shuffle=True)
            te_load = DataLoader(te_set, batch_size=1, shuffle=False)

            # 2. Setup Model & Loss
            model = ConfigurableUNet(depth=depth).to(DEVICE)
            optimizer = optim.Adam(model.parameters(), lr=LR)

            if loss_type == 'cosine':
                criterion = lambda p, t, m: (1 - (p*t).sum(1, keepdim=True) * m).sum() / (m.sum() + 1e-8)
            else:
                # MSE on Sin/Cos components
                criterion = lambda p, t, m: ((p-t)**2 * m).sum() / (m.sum()*2 + 1e-8)

            # 3. Train
            losses = []
            for epoch in range(EPOCHS):
                model.train()
                ep_loss = 0
                for img, tgt, msk, _ in tr_load:
                    img, tgt, msk = img.to(DEVICE), tgt.to(DEVICE), msk.to(DEVICE)
                    optimizer.zero_grad()
                    loss = criterion(model(img), tgt, msk)
                    loss.backward()
                    optimizer.step()
                    ep_loss += loss.item()
                losses.append(ep_loss/len(tr_load))

            # Save Loss Curve
            plt.figure()
            plt.plot(losses)
            plt.title(f"Loss: {model_name}")
            plt.xlabel("Epoch")
            plt.ylabel("Loss")
            plt.savefig(os.path.join(SAVE_DIR, f"{model_name}_Loss.png"))
            plt.close()

            # 4. Evaluate & Visualize
            metrics = evaluate_and_plot(model, te_load, model_name)
            FINAL_RESULTS[model_name] = metrics

            print(f"   -> MAE: {metrics[0]:.2f}° | Success: {metrics[1]:.1f}% | Spread: {metrics[2]:.2f}")
            run_count += 1

In [ ]:
# 6. FINAL SUMMARY GENERATION
print("\n--- Final Reports ---")

# Save CSV
df_results = pd.DataFrame.from_dict(FINAL_RESULTS, orient='index',
                                  columns=['MAE', 'Success_Rate', 'Spread_Ratio', 'Bias'])
csv_path = os.path.join(SAVE_DIR, "All_Models_Comparison_Metrics.csv")
df_results.to_csv(csv_path)
print(f"Metrics CSV saved to: {csv_path}")

# Generate Comparison Plots
names = list(FINAL_RESULTS.keys())
maes = [FINAL_RESULTS[n][0] for n in names]
successs = [FINAL_RESULTS[n][1] for n in names]
spreads = [FINAL_RESULTS[n][2] for n in names]

fig, ax = plt.subplots(1, 3, figsize=(24, 6))

# Plot 1: MAE
ax[0].bar(names, maes, color='salmon', edgecolor='black')
ax[0].set_title("Mean Angular Error (Lower is Better)")
ax[0].set_ylabel("Degrees")
ax[0].tick_params(axis='x', rotation=90)
ax[0].grid(axis='y', alpha=0.3)

# Plot 2: Success Rate
ax[1].bar(names, successs, color='skyblue', edgecolor='black')
ax[1].set_title("Success Rate (<15° error) (Higher is Better)")
ax[1].set_ylabel("Percentage %")
ax[1].tick_params(axis='x', rotation=90)
ax[1].grid(axis='y', alpha=0.3)

# Plot 3: Spread Ratio
ax[2].plot(names, spreads, marker='o', linestyle='-', color='purple', linewidth=2)
ax[2].axhline(1.0, color='green', linestyle='--', label='Ideal (1.0)')
ax[2].axhline(0.1, color='red', linestyle='--', label='Collapse (<0.1)')
ax[2].set_title("Spread Ratio (Collapse Detector)")
ax[2].set_ylabel("Ratio")
ax[2].tick_params(axis='x', rotation=90)
ax[2].legend()
ax[2].grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig(os.path.join(SAVE_DIR, "All_Models_Comparison_Plots.png"))
plt.show()

print("Complete.")

### Adjustable configuration for optimal model (High contrast = True, Cosine Loss Function, Shallow 2-layer U-net)

In [ ]:
import os
import glob
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
import matplotlib.pyplot as plt
import tifffile as tiff
from skimage import exposure

# Fixed Settings
BATCH_SIZE = 16
LR = 5e-4
EPOCHS = 25
REGION_SIZE = 100
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'

# Paths
ORIGINAL_DATA_DIR = '/content/drive/Shareddrives/ENM 5310 Group Project/Training Data'
AUGMENTED_DATA_DIR = '/content/drive/Shareddrives/ENM 5310 Group Project/Augmented Data'

# ==========================================
# 1. DATASET
# ==========================================
class CollagenDataset(Dataset):
    def __init__(self, image_files, csv_df, use_clahe):
        self.image_files = image_files
        self.csv_df = csv_df
        self.use_clahe = use_clahe
        self.csv_df['clean_filename'] = self.csv_df.iloc[:, 0].apply(lambda x: os.path.basename(str(x)).strip())
        self.img_transform = transforms.Compose([transforms.ToTensor()])

    def __len__(self): return len(self.image_files)

    def _generate_gt_map(self, filename, shape):
        clean_name = os.path.basename(filename).strip()
        file_data = self.csv_df[self.csv_df['clean_filename'] == clean_name]

        acc_u = np.zeros(shape, dtype=np.float32)
        acc_v = np.zeros(shape, dtype=np.float32)
        mask = np.zeros(shape, dtype=np.float32)
        half = REGION_SIZE // 2

        for _, row in file_data.iterrows():
            pos_str = row.iloc[1]
            try:
                coords = [int(c) for c in pos_str.replace('(', '').replace(')', '').split(',')]
                cx, cy = coords[0], coords[1]
                y0, y1 = cy - half, cy + half
                x0, x1 = cx - half, cx + half
                if not (x0>=0 and x1<=shape[1] and y0>=0 and y1<=shape[0]): continue

                if 'u' in row: u = float(row['u']); v = float(row['v'])
                else: u = float(row.iloc[2]); v = float(row.iloc[3])

                mag = np.sqrt(u**2 + v**2)
                if mag > 1e-6: u/=mag; v/=mag
                if u < 0: u = -u; v = -v

                acc_u[y0:y1, x0:x1] += u
                acc_v[y0:y1, x0:x1] += v
                mask[y0:y1, x0:x1] = 1.0
            except: continue

        theta = np.arctan(acc_v / (acc_u + 1e-8))
        target_sin = np.sin(2 * theta); target_cos = np.cos(2 * theta)
        target_sin[mask == 0] = 0; target_cos[mask == 0] = 0

        gt_map = np.stack([target_sin, target_cos], axis=0)
        return gt_map, np.expand_dims(mask, axis=0)

    def __getitem__(self, idx):
        path = self.image_files[idx]
        image = tiff.imread(path).astype(np.float32)

        # --- EXPERIMENT VARIABLE A: PREPROCESSING ---
        if image.max() > image.min():
            image = (image - image.min()) / (image.max() - image.min())

        if self.use_clahe:
            image = exposure.equalize_adapthist(image, kernel_size=None, clip_limit=0.03)
            image = image.astype(np.float32)
        # ---------------------------------------------

        if len(image.shape) == 2: image = np.expand_dims(image, axis=0)
        img_t = torch.from_numpy(image)
        gt, mask = self._generate_gt_map(os.path.basename(path), (image.shape[1], image.shape[2]))
        return img_t, torch.from_numpy(gt), torch.from_numpy(mask), idx

# ==========================================
# 2. MODELS
# ==========================================
class ResidualBlock(nn.Module):
    def __init__(self, in_c, out_c):
        super().__init__()
        self.conv = nn.Sequential(nn.BatchNorm2d(in_c), nn.ReLU(),
                                  nn.Conv2d(in_c, out_c, 3, 1, 1),
                                  nn.BatchNorm2d(out_c), nn.ReLU(),
                                  nn.Conv2d(out_c, out_c, 3, 1, 1))
        self.short = nn.Conv2d(in_c, out_c, 1) if in_c != out_c else nn.Identity()
    def forward(self, x): return self.short(x) + self.conv(x)

class ConfigurableUNet(nn.Module):
    def __init__(self, depth='deep'):
        super().__init__()
        self.depth = depth
        self.inc = nn.Sequential(nn.Conv2d(1, 32, 3, 1, 1), nn.ReLU(), nn.Conv2d(32, 32, 3, 1, 1))

        # --- EXPERIMENT VARIABLE C: DEPTH ---
        self.down1 = nn.Sequential(nn.MaxPool2d(2), ResidualBlock(32, 64))
        self.down2 = nn.Sequential(nn.MaxPool2d(2), ResidualBlock(64, 128))

        if self.depth == 'deep':
            self.down3 = nn.Sequential(nn.MaxPool2d(2), ResidualBlock(128, 256))
            self.up2 = nn.ConvTranspose2d(256, 128, 2, 2)
            self.conv2 = ResidualBlock(256, 128)

        self.up3 = nn.ConvTranspose2d(128, 64, 2, 2)
        self.conv3 = ResidualBlock(128, 64)
        self.up4 = nn.ConvTranspose2d(64, 32, 2, 2)
        self.conv4 = ResidualBlock(64, 32)
        self.outc = nn.Conv2d(32, 2, 1)

    def forward(self, x):
        x1 = self.inc(x)
        x2 = self.down1(x1)
        x3 = self.down2(x2)

        if self.depth == 'deep':
            x4 = self.down3(x3)
            # Up
            x = self.up2(x4)
            dY = x3.size(2)-x.size(2); dX = x3.size(3)-x.size(3)
            x = nn.functional.pad(x, [dX//2, dX-dX//2, dY//2, dY-dY//2])
            x = self.conv2(torch.cat([x3, x], 1))
        else:
            x = x3

        x = self.up3(x)
        dY = x2.size(2)-x.size(2); dX = x2.size(3)-x.size(3)
        x = nn.functional.pad(x, [dX//2, dX-dX//2, dY//2, dY-dY//2])
        x = self.conv3(torch.cat([x2, x], 1))

        x = self.up4(x)
        dY = x1.size(2)-x.size(2); dX = x1.size(3)-x.size(3)
        x = nn.functional.pad(x, [dX//2, dX-dX//2, dY//2, dY-dY//2])
        x = self.conv4(torch.cat([x1, x], 1))

        # Normalize to Unit Vector
        return F.normalize(self.outc(x), p=2, dim=1)

# ==========================================
# 3. METRIC CALCULATION HELPERS
# ==========================================
def get_shortest_angle_diff(angle1, angle2):
    """
    Calculates the smallest difference between two angles in degrees,
    accounting for 180-degree (pi radian) symmetry of fibers.
    """
    # Convert input to radians if they are in degrees, or handle as radians
    # Assuming inputs are already in degrees for this helper
    diff = np.abs(angle1 - angle2)
    diff = diff % 180  # Normalize to [0, 180]
    diff = np.minimum(diff, 180 - diff) # Distance to nearest 180 multiple
    return diff

def calculate_metrics(gt_angles, pred_angles):
    """
    Computes the 4 key metrics from raw angle arrays (in degrees).
    """
    # 1. Angular Errors (Absolute)
    errors = get_shortest_angle_diff(gt_angles, pred_angles)
    mae = np.mean(errors)

    # 2. Success Rate (Tolerance < 15 degrees)
    success_rate = np.mean(errors < 15.0) * 100

    # 3. Spread Ratio (Pred Std / GT Std)
    # Measures if the model has collapsed (Ratio < 0.1)
    std_gt = np.std(gt_angles)
    std_pred = np.std(pred_angles)
    spread_ratio = std_pred / (std_gt + 1e-8)

    # 4. Bias (Signed Mean Error)
    # Simple mean difference (approximate for circular data but useful for trend)
    bias = np.mean(pred_angles) - np.mean(gt_angles)

    return {
        "MAE (deg)": mae,
        "Success Rate (%)": success_rate,
        "Spread Ratio": spread_ratio,
        "Bias (deg)": bias
    }

# ==========================================
# 4. COMPREHENSIVE EVALUATION FUNCTION
# ==========================================
def evaluate_model_comprehensive(model, loader, device='cuda', num_show=3):
    model.eval()

    all_gt_angles = []
    all_pred_angles = []

    print("\n--- Starting Comprehensive Evaluation ---")

    with torch.no_grad():
        for i, (images, _, _, indices) in enumerate(loader):
            images_dev = images.to(device)
            outputs = model(images_dev)

            # Extract Batch Info
            idx = indices[0].item()
            path = loader.dataset.image_files[idx]
            filename = os.path.basename(path)

            # Unpack Prediction Map (Sin/Cos -> Angle)
            sin_map = outputs[0, 0].cpu().numpy()
            cos_map = outputs[0, 1].cpu().numpy()
            pred_theta_map = 0.5 * np.arctan2(sin_map, cos_map) # Radians [-pi/2, pi/2]

            # Get Ground Truth Vectors from CSV
            file_data = loader.dataset.csv_df[loader.dataset.csv_df['clean_filename'] == filename]

            X_list, Y_list = [], []
            U_gt_list, V_gt_list = [], []
            U_pd_list, V_pd_list = [], []

            h, w = pred_theta_map.shape

            for _, row in file_data.iterrows():
                try:
                    # Parse Grid Position
                    c = [int(x) for x in row.iloc[1].replace('(', '').replace(')', '').split(',')]
                    cx, cy = c[0], c[1]

                    # Bounds Check
                    half = 100 // 2 # Region Size
                    if not (cx-half >= 0 and cx+half <= w and cy-half >= 0 and cy+half <= h):
                        continue

                    # 1. Process Ground Truth
                    if 'u' in row: u, v = float(row['u']), float(row['v'])
                    else: u, v = float(row.iloc[2]), float(row.iloc[3])

                    # Normalize & Force Right Half-Plane (Symmetry)
                    mag = np.sqrt(u**2 + v**2)
                    if mag > 1e-6: u /= mag; v /= mag
                    if u < 0: u = -u; v = -v

                    gt_angle_deg = np.degrees(np.arctan2(v, u + 1e-8))

                    # 2. Process Prediction (Sample at center)
                    pred_rad = pred_theta_map[cy, cx]
                    pred_angle_deg = np.degrees(pred_rad)

                    pu, pv = np.cos(pred_rad), np.sin(pred_rad)

                    # Store for Metrics
                    all_gt_angles.append(gt_angle_deg)
                    all_pred_angles.append(pred_angle_deg)

                    # Store for Plotting
                    X_list.append(cx); Y_list.append(cy)
                    U_gt_list.append(u); V_gt_list.append(v)
                    U_pd_list.append(pu); V_pd_list.append(pv)

                except: continue

            # --- Visualization (Only show first 'num_show' images) ---
            if i < num_show:
                img_np = images[0, 0].cpu().numpy()
                fig, ax = plt.subplots(1, 2, figsize=(18, 5))

                # Ground Truth
                ax[0].imshow(img_np, cmap='gray')
                ax[0].quiver(X_list, Y_list, U_gt_list, V_gt_list, color='lime', scale=20, pivot='mid', headaxislength=0)
                ax[0].set_title(f"Ground Truth: {filename}")

                # Prediction
                ax[1].imshow(img_np, cmap='gray')
                ax[1].quiver(X_list, Y_list, U_pd_list, V_pd_list, color='red', scale=20, pivot='mid', headaxislength=0)
                ax[1].set_title(f"Prediction: {filename}")

                plt.show()

    # --- Final Metric Calculation ---
    results = calculate_metrics(np.array(all_gt_angles), np.array(all_pred_angles))

    print("\n" + "="*40)
    print(f"FINAL RESULTS (Evaluated on {len(all_gt_angles)} vectors)")
    print("="*40)
    print(f"Mean Angular Error:   {results['MAE (deg)']:.2f}°  (Lower is better)")
    print(f"Success Rate (<15°):  {results['Success Rate (%)']:.1f}%   (Higher is better)")
    print(f"Spread Ratio:         {results['Spread Ratio']:.2f}    (Target ~1.0. <0.1 means collapse)")
    print(f"Bias:                 {results['Bias (deg)']:.2f}°   (Signed error)")
    print("="*40 + "\n")

    return results

# ==========================================
# 5. EXPERIMENT PLAYGROUND
# ==========================================
# Variable A: Preprocessing
USE_CLAHE = True       # True = Enhance contrast. False = Raw intensity.

# Variable B: Loss Function
# 'cosine' = Vector similarity (Smooth). 'mse' = Direct angle error (Harder).
LOSS_TYPE = 'cosine'

# Variable C: Model Complexity
MODEL_DEPTH = 'shallow'   # 'deep' = Standard 4-level U-Net. 'shallow' = Fast 2-level U-Net.

# 6. EXECUTION
# ==========================================
# Load
try: df_tr = pd.read_csv(os.path.join(AUGMENTED_DATA_DIR, 'augmented_labels.csv'))
except: df_tr = pd.DataFrame()
tr_files = sorted(glob.glob(os.path.join(AUGMENTED_DATA_DIR, '*.tiff')))

try: df_te = pd.read_csv(os.path.join(ORIGINAL_DATA_DIR, 'fibril_orientation_2d_results.csv'))
except: df_te = pd.DataFrame()
all_orig = sorted(glob.glob(os.path.join(ORIGINAL_DATA_DIR, 'training*.tiff')))
import re
te_files = [f for f in all_orig if int(re.findall(r'\d+', os.path.basename(f))[0]) >= 10]

tr_set = CollagenDataset(tr_files, df_tr, use_clahe=USE_CLAHE)
te_set = CollagenDataset(te_files, df_te, use_clahe=USE_CLAHE)
tr_load = DataLoader(tr_set, batch_size=BATCH_SIZE, shuffle=True)
te_load = DataLoader(te_set, batch_size=1, shuffle=False)

print(f"--- RUNNING EXPERIMENT ---")
print(f"CLAHE: {USE_CLAHE} | LOSS: {LOSS_TYPE} | DEPTH: {MODEL_DEPTH}")
print(f"Training on {len(tr_files)} images...")

model = ConfigurableUNet(depth=MODEL_DEPTH).to(DEVICE)
optimizer = optim.Adam(model.parameters(), lr=LR)

# --- EXPERIMENT VARIABLE B: LOSS ---
if LOSS_TYPE == 'cosine':
    # 1 - Cosine Similarity
    criterion = lambda p, t, m: (1 - (p*t).sum(1, keepdim=True) * m).sum() / (m.sum() + 1e-8)
else:
    # MSE Loss
    criterion = lambda p, t, m: ((p-t)**2 * m).sum() / (m.sum()*2 + 1e-8)

losses = []
for epoch in range(EPOCHS):
    model.train()
    ep_loss = 0
    for img, tgt, msk, _ in tr_load:
        img, tgt, msk = img.to(DEVICE), tgt.to(DEVICE), msk.to(DEVICE)
        optimizer.zero_grad()
        loss = criterion(model(img), tgt, msk)
        loss.backward()
        optimizer.step()
        ep_loss += loss.item()
    print(f"Ep {epoch+1}: Loss={ep_loss/len(tr_load):.4f}")
    losses.append(ep_loss/len(tr_load))

plt.plot(losses); plt.title(f"Loss Curve ({LOSS_TYPE})"); plt.show()

# Visualizer (No Changes)
def viz(model, loader):
    model.eval()
    with torch.no_grad():
        for _, (images, _, _, indices) in enumerate(loader):
            out = model(images.to(DEVICE))
            path = loader.dataset.image_files[indices[0].item()]
            name = os.path.basename(path)

            sin_map = out[0, 0].cpu().numpy(); cos_map = out[0, 1].cpu().numpy()
            p_theta = 0.5 * np.arctan2(sin_map, cos_map)

            # Extract Grid
            file_data = loader.dataset.csv_df[loader.dataset.csv_df['clean_filename'] == name]
            X, Y, Ug, Vg, Up, Vp = [], [], [], [], [], []
            h, w = p_theta.shape

            for _, row in file_data.iterrows():
                try:
                    c = [int(x) for x in row.iloc[1].replace('(','').replace(')','').split(',')]
                    cx, cy = c[0], c[1]
                    if not (0<=cx<w and 0<=cy<h): continue

                    # GT
                    u = float(row.iloc[2]); v = float(row.iloc[3])
                    m = np.sqrt(u**2+v**2)
                    if m > 1e-6: u/=m; v/=m

                    # Pred
                    th = p_theta[cy, cx]
                    pu, pv = np.cos(th), np.sin(th)

                    X.append(cx); Y.append(cy)
                    Ug.append(u); Vg.append(v)
                    Up.append(pu); Vp.append(pv)
                except: continue

            img_np = images[0, 0].cpu().numpy()
            f, ax = plt.subplots(1, 2, figsize=(15, 5))
            ax[0].imshow(img_np, cmap='gray'); ax[0].quiver(X, Y, Ug, Vg, color='lime', scale=20, pivot='mid')
            ax[0].set_title("Ground Truth")
            ax[1].imshow(img_np, cmap='gray'); ax[1].quiver(X, Y, Up, Vp, color='red', scale=20, pivot='mid')
            ax[1].set_title(f"Prediction ({LOSS_TYPE}, {MODEL_DEPTH})")
            plt.show()

if len(te_files) > 0: viz(model, te_load)

New code for comparing different ways to augment the data

In [ ]:
import os
import glob
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
import matplotlib.pyplot as plt
import seaborn as sns
import tifffile as tiff
from skimage import exposure, restoration
from sklearn.model_selection import train_test_split
from tqdm import tqdm
import random

# ==========================================
# 1. CONFIGURATION
# ==========================================
# Adjust these paths to your specific Drive folder structure
BASE_DIR = '/content/drive/Shareddrives/ENM 5310 Group Project'
ORIGINAL_DATA_DIR = os.path.join(BASE_DIR, 'Training Data')
AUG_OUTPUT_DIR = os.path.join(BASE_DIR, 'Experiment_Data_v2')
MODEL_SAVE_DIR = os.path.join(BASE_DIR, 'Models_v2')
CSV_FILENAME = 'fibril_orientation_2d_results.csv'

# Hyperparameters
BATCH_SIZE = 16
LR = 5e-4
EPOCHS = 15 # Reduced slightly for speed in this demo
REGION_SIZE = 100 # This is for GT map generation
FIXED_IMAGE_SIZE = 256 # Define a fixed square size for image inputs to the model
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'

for d in [AUG_OUTPUT_DIR, MODEL_SAVE_DIR]:
    if not os.path.exists(d):
        os.makedirs(d)


In [ ]:
# ==========================================
# 2. DATA AUGMENTATION FUNCTIONS
# ==========================================

def parse_pos(pos_str):
    """Parses (y, x, 0) string from CSV."""
    try:
        coords = [int(c) for c in pos_str.replace('(', '').replace(')', '').split(',')]
        return coords[0], coords[1] # y, x
    except:
        return None, None

def generate_rotated_data(file_list, df, output_subdir, fixed_size=FIXED_IMAGE_SIZE):
    """Generates rotated and center-cropped images and updates vector labels."""
    save_path = os.path.join(AUG_OUTPUT_DIR, output_subdir)
    if not os.path.exists(save_path): os.makedirs(save_path)

    new_csv_rows = []
    print(f"Generating Rotated Data in {output_subdir}...")

    for file_path in tqdm(file_list):
        filename = os.path.basename(file_path)
        base_name = os.path.splitext(filename)[0]
        try: img = tiff.imread(file_path)
        except: continue

        orig_h, orig_w = img.shape
        img_labels = df[df['clean_filename'] == filename]

        for angle in [0, 90]: # Using 0 and 90 degree rotations
            # 1. Rotate Image
            k = angle // 90
            rot_img = np.rot90(img, k=k)
            curr_h, curr_w = rot_img.shape

            # Center crop the rotated image to a fixed size
            if curr_h < fixed_size or curr_w < fixed_size:
                # If image is smaller than fixed_size, pad it
                pad_h_before = (fixed_size - curr_h) // 2
                pad_h_after = fixed_size - curr_h - pad_h_before
                pad_w_before = (fixed_size - curr_w) // 2
                pad_w_after = fixed_size - curr_w - pad_w_before
                cropped_img = np.pad(rot_img, ((pad_h_before, pad_h_after), (pad_w_before, pad_w_after)), mode='constant')
            else:
                # Center crop
                y_start = (curr_h - fixed_size) // 2
                x_start = (curr_w - fixed_size) // 2
                cropped_img = rot_img[y_start:y_start+fixed_size, x_start:x_start+fixed_size]

            # Save the cropped image for this rotation
            new_img_name = f"{base_name}_rot{angle}.tiff"
            tiff.imwrite(os.path.join(save_path, new_img_name), cropped_img)

            # 2. Rotate and Adjust Vectors
            # Calculate the offsets for cropping from the rotated image
            crop_offset_y = (curr_h - fixed_size) // 2
            crop_offset_x = (curr_w - fixed_size) // 2

            for _, row in img_labels.iterrows():
                py, px = parse_pos(row.iloc[1]) # original coords relative to original image
                if px is None: continue

                # Get coords and vectors in the *rotated* image frame (before cropping)
                rx_rot, ry_rot = px, py
                ru_rot, rv_rot = float(row.iloc[2]), float(row.iloc[3])

                if angle == 90:
                    # For 90 deg CCW rotation:
                    # New X = Old Y
                    # New Y = (Original Width - 1) - Old X
                    # New U = Old V
                    # New V = -Old U
                    rx_rot = py
                    ry_rot = (orig_w - 1) - px
                    ru_rot = float(row.iloc[3])
                    rv_rot = -float(row.iloc[2])

                # Now, adjust these (rx_rot, ry_rot) for the cropping to fixed_size
                # These coordinates are relative to the top-left of the rotated image
                rx_cropped = rx_rot - crop_offset_x
                ry_cropped = ry_rot - crop_offset_y

                # Only add label if it falls within the *fixed_size* cropped region
                if 0 <= rx_cropped < fixed_size and 0 <= ry_cropped < fixed_size:
                    new_csv_rows.append({
                        'filename': new_img_name,
                        'pos': f"({int(ry_cropped)},{int(rx_cropped)},0)",
                        'u': ru_rot,
                        'v': rv_rot
                    })

    out_df = pd.DataFrame(new_csv_rows)
    out_df.to_csv(os.path.join(save_path, 'labels.csv'), index=False)
    return save_path

def generate_blurred_data(file_list, df, output_subdir, fixed_size=FIXED_IMAGE_SIZE):
    """
    Generates Bilateral Filtered images and ensures fixed output size.
    Note: Blurring does NOT change vector direction or coordinates.
    """
    save_path = os.path.join(AUG_OUTPUT_DIR, output_subdir)
    if not os.path.exists(save_path): os.makedirs(save_path)

    new_csv_rows = []
    print(f"Generating Blurred Data in {output_subdir}...")

    for file_path in tqdm(file_list):
        filename = os.path.basename(file_path)
        base_name = os.path.splitext(filename)[0]
        try: img = tiff.imread(file_path)
        except: continue

        orig_h, orig_w = img.shape

        # Normalize for skimage float requirement
        img_norm = (img - img.min()) / (img.max() - img.min() + 1e-8) # Added epsilon to prevent division by zero

        # Apply Bilateral Filter
        blurred_img = restoration.denoise_bilateral(
            img_norm, sigma_color=0.1, sigma_spatial=5, channel_axis=None
        )

        # Rescale back to original range (assuming original was 16-bit tiff)
        blurred_img = (blurred_img * 65535).astype(np.uint16)

        # Center crop the blurred image to a fixed size
        curr_h, curr_w = blurred_img.shape
        if curr_h < fixed_size or curr_w < fixed_size:
            pad_h_before = (fixed_size - curr_h) // 2
            pad_h_after = fixed_size - curr_h - pad_h_before
            pad_w_before = (fixed_size - curr_w) // 2
            pad_w_after = fixed_size - curr_w - pad_w_before
            cropped_img = np.pad(blurred_img, ((pad_h_before, pad_h_after), (pad_w_before, pad_w_after)), mode='constant')
        else:
            y_start = (curr_h - fixed_size) // 2
            x_start = (curr_w - fixed_size) // 2
            cropped_img = blurred_img[y_start:y_start+fixed_size, x_start:x_start+fixed_size]

        new_img_name = f"blur_{base_name}.tiff"
        tiff.imwrite(os.path.join(save_path, new_img_name), cropped_img)

        # Copy labels exactly (vectors don't change with blur), but adjust for crop
        img_labels = df[df['clean_filename'] == filename]

        # Calculate the offsets for cropping from the original image (which matches blurred_img size before crop)
        crop_offset_y = (orig_h - fixed_size) // 2
        crop_offset_x = (orig_w - fixed_size) // 2

        for _, row in img_labels.iterrows():
            new_row = row.to_dict()
            new_row['filename'] = new_img_name
            # Ensure u/v are explicitly set if they were positional in original csv
            if 'u' not in new_row:
                new_row['u'] = row.iloc[2]
                new_row['v'] = row.iloc[3]

            py, px = parse_pos(row.iloc[1]) # original coords relative to original image
            if px is None: continue

            # Adjust coordinates for the cropping
            px_cropped = px - crop_offset_x
            py_cropped = py - crop_offset_y

            # Only add label if it falls within the *fixed_size* cropped region
            if 0 <= px_cropped < fixed_size and 0 <= py_cropped < fixed_size:
                new_row['pos'] = f"({int(py_cropped)},{int(px_cropped)},0)"
                new_csv_rows.append(new_row)

    out_df = pd.DataFrame(new_csv_rows)
    out_df.to_csv(os.path.join(save_path, 'labels.csv'), index=False)
    return save_path


In [ ]:
# ==========================================
# 3. DATASET & MODEL CLASSES
# ==========================================
class CollagenDataset(Dataset):
    def __init__(self, file_list, df, fixed_size=FIXED_IMAGE_SIZE):
        self.files = file_list
        self.df = df
        self.fixed_size = fixed_size
        # Ensure filename matching column exists
        if 'clean_filename' not in self.df.columns:
            self.df['clean_filename'] = self.df['filename']

    def __len__(self): return len(self.files)

    def __getitem__(self, idx):
        path = self.files[idx]
        fname = os.path.basename(path)

        img = tiff.imread(path).astype(np.float32)
        # Standardize Input
        if img.max() > img.min():
            img = (img - img.min()) / (img.max() - img.min() + 1e-8) # Added epsilon

        # Ensure image is fixed_size. This handles 'Unmodified' images and ensures consistency
        # if there were any issues with previous augmentation steps.
        curr_h, curr_w = img.shape
        if curr_h < self.fixed_size or curr_w < self.fixed_size:
            # Pad if image is smaller than fixed_size
            pad_h_before = (self.fixed_size - curr_h) // 2
            pad_h_after = self.fixed_size - curr_h - pad_h_before
            pad_w_before = (self.fixed_size - curr_w) // 2
            pad_w_after = self.fixed_size - curr_w - pad_w_before
            img_processed = np.pad(img, ((pad_h_before, pad_h_after), (pad_w_before, pad_w_after)), mode='constant')
        else:
            # Center crop if image is larger or equal to fixed_size
            y_start = (curr_h - self.fixed_size) // 2
            x_start = (curr_w - self.fixed_size) // 2
            img_processed = img[y_start:y_start+self.fixed_size, x_start:x_start+self.fixed_size]

        img_processed = np.expand_dims(img_processed, 0)

        # GT Generation
        h, w = img_processed.shape[1], img_processed.shape[2] # Use processed image dimensions for GT map
        file_data = self.df[self.df['clean_filename'] == fname]

        acc_u = np.zeros((h, w), dtype=np.float32)
        acc_v = np.zeros((h, w), dtype=np.float32)
        mask = np.zeros((h, w), dtype=np.float32)

        # `pos` values in the CSVs are now assumed to be adjusted to the `fixed_size` image due to the changes in `generate_rotated_data` and `generate_blurred_data`.
        for _, row in file_data.iterrows():
            try:
                p_str = str(row['pos']) if 'pos' in row else row.iloc[1]
                c = [int(x) for x in p_str.replace('(', '').replace(')', '').split(',')]
                cy, cx = c[0], c[1]

                # REGION_SIZE is 100, so half is 50
                start_y, end_y = max(0, cy-REGION_SIZE//2), min(h, cy+REGION_SIZE//2)
                start_x, end_x = max(0, cx-REGION_SIZE//2), min(w, cx+REGION_SIZE//2)

                u = float(row['u']) if 'u' in row else float(row.iloc[2])
                v = float(row['v']) if 'v' in row else float(row.iloc[3])

                acc_u[start_y:end_y, start_x:end_x] += u
                acc_v[start_y:end_y, start_x:end_x] += v
                mask[start_y:end_y, start_x:end_x] = 1.0
            except: continue

        theta = np.arctan2(acc_v, acc_u + 1e-8)
        gt_sin = np.sin(2*theta) * mask
        gt_cos = np.cos(2*theta) * mask

        return torch.from_numpy(img_processed), torch.from_numpy(np.stack([gt_sin, gt_cos])), torch.from_numpy(mask)

class SimpleUNet(nn.Module):
    def __init__(self):
        super().__init__()
        # Shallow U-Net for speed
        self.enc1 = nn.Sequential(nn.Conv2d(1, 32, 3, 1, 1), nn.ReLU())
        self.enc2 = nn.Sequential(nn.Conv2d(32, 64, 3, 1, 1), nn.ReLU())
        self.pool = nn.MaxPool2d(2)

        self.mid = nn.Sequential(nn.Conv2d(64, 128, 3, 1, 1), nn.ReLU())

        self.up2 = nn.ConvTranspose2d(128, 64, 2, 2)
        self.dec2 = nn.Conv2d(128, 64, 3, 1, 1)
        self.up1 = nn.ConvTranspose2d(64, 32, 2, 2)
        self.dec1 = nn.Conv2d(64, 32, 3, 1, 1)
        self.out = nn.Conv2d(32, 2, 1) # Sin, Cos

    def forward(self, x):
        e1 = self.enc1(x)
        e2 = self.enc2(self.pool(e1))

        m = self.mid(self.pool(e2))

        d2 = self.dec2(torch.cat([self.up2(m), e2], 1))
        d1 = self.dec1(torch.cat([self.up1(d2), e1], 1))

        return F.normalize(self.out(d1), p=2, dim=1)


In [ ]:
# ==========================================
# 4. DATA GENERATION
# ==========================================
# Load Master CSV
csv_path = os.path.join(ORIGINAL_DATA_DIR, CSV_FILENAME)
master_df = pd.read_csv(csv_path)
master_df['clean_filename'] = master_df.iloc[:, 0].apply(lambda x: os.path.basename(str(x)).strip())

# Get all unique Tiff files
all_tiffs = sorted(glob.glob(os.path.join(ORIGINAL_DATA_DIR, '*.tiff')))
train_files, test_files = train_test_split(all_tiffs, test_size=0.2, random_state=42)

print(f"Split: {len(train_files)} Train, {len(test_files)} Test")

# --- Generate 3 Training Sets ---
# 1. Unmodified (Images will be cropped to FIXED_IMAGE_SIZE by CollagenDataset)
train_sets = {'Unmodified': (train_files, master_df)}

# 2. Rotated
rot_path = generate_rotated_data(train_files, master_df, 'Train_Rotated', fixed_size=FIXED_IMAGE_SIZE)
rot_df = pd.read_csv(os.path.join(rot_path, 'labels.csv'))
rot_files = sorted(glob.glob(os.path.join(rot_path, '*.tiff')))
train_sets['Rotated'] = (rot_files, rot_df)

# 3. Blurred
blur_path = generate_blurred_data(train_files, master_df, 'Train_Blurred', fixed_size=FIXED_IMAGE_SIZE)
blur_df = pd.read_csv(os.path.join(blur_path, 'labels.csv'))
blur_files = sorted(glob.glob(os.path.join(blur_path, '*.tiff')))
train_sets['Blurred'] = (blur_files, blur_df)


# --- Generate 3 Test Sets ---
test_sets = {'Unmodified': (test_files, master_df)}

rot_test_path = generate_rotated_data(test_files, master_df, 'Test_Rotated', fixed_size=FIXED_IMAGE_SIZE)
rot_test_df = pd.read_csv(os.path.join(rot_test_path, 'labels.csv'))
rot_test_files = sorted(glob.glob(os.path.join(rot_test_path, '*.tiff')))
test_sets['Rotated'] = (rot_test_files, rot_test_df)

blur_test_path = generate_blurred_data(test_files, master_df, 'Test_Blurred', fixed_size=FIXED_IMAGE_SIZE)
blur_test_df = pd.read_csv(os.path.join(blur_test_path, 'labels.csv'))
blur_test_files = sorted(glob.glob(os.path.join(blur_test_path, '*.tiff')))
test_sets['Blurred'] = (blur_test_files, blur_test_df)


FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/Shareddrives/ENM 5310 Group Project/Training Data/fibril_orientation_2d_results.csv'

In [ ]:
# ==========================================
# 5. MAIN TRAINING & EVALUATION LOOP
# ==========================================
matrix_results = np.zeros((3, 3)) # Rows: Models, Cols: TestSets
model_names = ['Unmodified', 'Rotated', 'Blurred']
trained_models = []

# --- A. TRAINING PHASE ---
for i, m_name in enumerate(model_names):
    print(f"\nTraining Model: {m_name}...")

    files, df = train_sets[m_name]
    dataset = CollagenDataset(files, df, fixed_size=FIXED_IMAGE_SIZE)
    loader = DataLoader(dataset, batch_size=BATCH_SIZE, shuffle=True)

    model = SimpleUNet().to(DEVICE)
    optimizer = optim.Adam(model.parameters(), lr=LR)

    # Cosine Loss
    criterion = lambda p, t, m: (1 - (p*t).sum(1, keepdim=True) * m).sum() / (m.sum() + 1e-8)

    model.train()
    for ep in range(EPOCHS):
        total_loss = 0
        for img, tgt, msk in loader:
            img, tgt, msk = img.to(DEVICE), tgt.to(DEVICE), msk.to(DEVICE)
            optimizer.zero_grad()
            loss = criterion(model(img), tgt, msk)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()

    trained_models.append(model)
    torch.save(model.state_dict(), os.path.join(MODEL_SAVE_DIR, f"model_{m_name}.pth"))
    print(f"Finished {m_name}. Final Loss: {total_loss/len(loader):.4f}")

# --- B. EVALUATION PHASE (CROSS-MATRIX) ---
def get_mae(model, loader):
    model.eval()
    errors = []
    with torch.no_grad():
        for img, tgt, msk in loader:
            img = img.to(DEVICE)
            out = model(img) # (B, 2, H, W)

            # Convert back to angles
            pred_ang = 0.5 * np.arctan2(out[:,0].cpu().numpy(), out[:,1].cpu().numpy())
            gt_ang = 0.5 * np.arctan2(tgt[:,0].numpy(), tgt[:,1].numpy())

            # Mask valid regions
            mask_np = msk.numpy() == 1

            diff = np.abs(np.degrees(pred_ang[mask_np]) - np.degrees(gt_ang[mask_np]))
            diff = diff % 180
            diff = np.minimum(diff, 180 - diff)
            errors.extend(diff)

    return np.mean(errors) if len(errors) > 0 else 90.0

print("\n--- Generating Cross-Evaluation Matrix ---")
for i, m_name in enumerate(model_names):     # Rows
    model = trained_models[i]
    for j, t_name in enumerate(model_names): # Cols
        files, df = test_sets[t_name]
        dset = CollagenDataset(files, df, fixed_size=FIXED_IMAGE_SIZE)
        # Use batch_size=4 for evaluation speed
        loader = DataLoader(dset, batch_size=4, shuffle=False)

        mae = get_mae(model, loader)
        matrix_results[i, j] = mae
        print(f"Model[{m_name}] on Test[{t_name}] -> MAE: {mae:.2f}")


In [ ]:
# ==========================================
# 6. VISUALIZATION (CONFUSION MATRIX)
# ==========================================
plt.figure(figsize=(8, 6))
sns.heatmap(matrix_results, annot=True, fmt=".2f", cmap="RdYlGn_r",
            xticklabels=model_names, yticklabels=model_names)
plt.title("Cross-Evaluation Matrix (MAE in Degrees)\nRows=Model Trained On, Cols=Tested On")
plt.xlabel("Test Data Type")
plt.ylabel("Training Data Type")
plt.tight_layout()
plt.savefig(os.path.join(BASE_DIR, 'Evaluation_Matrix.png'))
plt.show()


In [ ]:
import os
print(os.listdir('/content/drive/Shareddrives/ENM 5310 Group Project'))

['Models_v2', 'Experiment_Data_v2']
